## 12 - Evidência

### 12.1 - Interpretando as pontuações do SAT

- SAT: Um teste padronizado destinado a medir a preparação para o trabalho de nível universitário em matemática;

Se Alice obteve 780 e Bob obteve um 740 (de um possível 800), convém saber se essa diferença é uma evidência de que Alice está melhor preparada que Bob e qual é a força dessa evidência.

- Quão forte é a evidência de que Alice está melhor preparada que Bob?
- Finjo, temporariamente, que todas as perguntas do SAT são igualmente difíceis.

- `p_correct`: Probabilidade de responder corretamente a qualquer pergunta;

### 12.2 A escala

- Entender o processo de pontuação e dimensionamento do SAT;
- Pontuação bruta com base no número de perguntas corretas e incorretas;
- A pontuação é escalada no intervalo de 200 a 800;

- A pontuação bruta é o número de perguntas respondidas corretamente, menos uma penalidade de 1/4 de ponto para cada pergunta respondida incorretamente.


### 12.3 - A priori

- Podemos usar a distribuição de pontuações brutas para modelar a distribuição a priori de `p_correct`.

In [2]:
class Exam(object):

    def __init__(self):
        self.scale = ReadScale()
        scores = ReadRanks()
        score_pmf = thinkbayes.MakePmfFromDict(dict(scores))
        self.raw = self.ReverseScale(score_pmf)
        self.max_score = max(self.raw.Values())
        self.prior = DivideValues(self.raw, self.max_score)

![](p_corret.png)

- Distribuição a priori de `p_correct` para os participantes do teste SAT.
- Essa distribuição é aproximadamente gaussiana, mas é comprimida nos extremos
- o SAT tem mais poder de diferenciar entre os participantes do teste dentro de dois desvios padrão da média e menos poder fora desse intervalo.

In [4]:
import thinkbayes
class Sat(thinkbayes.Suite):

    def __init__(self, exam, score):
        thinkbayes.Suite.__init__(self)

        self.exam = exam
        self.score = score

        # start with the prior distribution
        for p_correct, prob in exam.prior.Items():
            self.Set(p_correct, prob)

        # update based on an exam score
        self.Update(score)
        
    def Likelihood(self, data, hypo):
        p_correct = hypo
        score = data

        k = self.exam.Reverse(score)
        n = self.exam.max_score
        like = thinkbayes.EvalBinomialPmf(k, n, p_correct)
        return like

- `hypo` é um valor hipotético de `p_correct`
- `data` é uma pontuação escalada.

- Pontuação bruta como o número de respostas corretas, ignorando a penalidade por respostas erradas;
- A verossimilhança é dada pela distribuição binomial, que calcula a probabilidade de k respostas corretas de n perguntas.

### 12.4 - Posteriori

![](posteriori_12.png)

- Distribuições posteriores de `p_correct` para Alice e Bob.

- Quão forte é a evidência de que Alice está melhor preparada que Bob?

Para formular a pergunta em termos de teste de hipótese bayesiana, defino duas hipóteses:

- A: `p_correct` mais alto para Alice do que para Bob.
- B: `p_correct` mais alto para Bob do que para Alice.

Para calcular a verossimilhança de **A** , podemos enumerar todos os pares de valores das distribuições posteriores e somar a probabilidade total dos casos em que `p_correct` é maior para Alice do que para Bob.

In [8]:
class TopLevel(thinkbayes.Suite):

    def Update(self, data):
        a_sat, b_sat = data

        a_like = thinkbayes.PmfProbGreater(a_sat, b_sat)
        b_like = thinkbayes.PmfProbLess(a_sat, b_sat)
        c_like = thinkbayes.PmfProbEqual(a_sat, b_sat)

        a_like += c_like / 2
        b_like += c_like / 2

        self.Mult('A', a_like)
        self.Mult('B', b_like)

        self.Normalize()

- **a_like** é a probabilidade total que `p_correct` é maior para Alice; 
- **b_like** é a probabilidade de que seja maior para Bob;
- **c_like** é a probabilidade de serem "iguais";

Se usarmos mais valores, c_like será menor e, no extremo, se `p_correct` for contínuo, c_like será zero;
- c_like, tipo de erro de arredondamento e divido igualmente entre a_like e b_like.

- A verossimilhança de A é 0,79 e a verossimilhança de B é 0,21. 
- A razão de verossimilhança (ou fator Bayes) é de 3.8.
    - O que significa que a evidência de que Alice é melhor que Bob em responder perguntas do SAT. 

A probabilidade de A é de 79%, o que significa que ainda há 21% de chance de Bob estar realmente melhor preparado.

### 12.5 - Um modelo melhor